# Automação Web e Busca de Informações com Python

 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
#- Dólar
#- Euro
#- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

#- Usaremos o selenium
#- Importante: baixar o webdriver

In [20]:
!pip install selenium

In [21]:
# BAIXAR o CROMEDRIVER para o SELENUIM FUNCIONAR.

from selenium import webdriver # serve para criar o navagador 
from selenium.webdriver.common.by import By # permite localizar elementos( os intens do site)
from selenium.webdriver.common.keys import Keys # permite clicar teclas do teclado 
navegador = webdriver.Chrome()

# PASSO 1 : ENTRAR NO GOOGLE
navegador.get("https://www.google.com.br/")

# PASSO 2 : PESQUISAR A COTAÇÃO DO DÓLAR 
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# PASSO 3 : PEGUAR A COTAÇÃO DO DÓLAR
cotacao_dolar = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)

# PASSO 4 : PEGAR A COTAÇÃO DO EURO
navegador.get("https://www.google.com.br/")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,'//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)

# PASSO 5 : PRGAR A COTAÇÃO DO OURO 
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH,'//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",",".")
print(cotacao_ouro)





5.055899999999999
5.274921588000001
296.32


###  atualiza a base de preços com as novas cotações

- Importando a base de dados

In [22]:
# ATUALIZAR A BASE DE DAOS COM AS NOVAS COTAÇÕES

- Atualizando os preços e o cálculo do Preço Final

In [23]:
import pandas as pd
tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


###  exportar a nova base de preços atualizada

In [24]:
# ATUALIZAR A COTAÇÃO DE ACORDO COM A MOEDA CORRESPONDENTE 
tabela.loc[tabela["Moeda"]=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"]=="Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"]=="Ouro", "Cotação"] = float(cotacao_ouro)
display(tabela)

# ATUALIZAR A PREÇO DE COMPRA = PREÇO ORIGINAL x COTAÇÃO 

tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# ATUALIZAR O PREÇO DE VENDA  = PREÇO DE COMPRA * MARGEM 

tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

                                           

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.055900,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.274922,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.055900,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.055900,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.274922,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.055900,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,296.320000,7000.00,1.15,8050.000


In [25]:
tabela.to_excel("Produtos Novos.xlsx", index=False)
navegador.quit()